<a href="https://colab.research.google.com/github/41371120h/PL-Repo.peng/blob/main/HW1_%E7%A4%BE%E5%9C%98%E6%94%B6%E6%94%AF%E7%B4%80%E9%8C%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 用於管理社團收支。
# 程式會自動從指定的 Google 試算表 (Google Sheets) 中讀取「收支紀錄」分頁，
# 使用者可以輸入日期、項目名稱、收入或支出金額、請款人、支付方式，以及是否已給錢。
# 系統會自動計算累積結餘，並將更新的結餘寫回試算表。
# 透過此程式，使用者能夠方便管理每日收支，隨時掌握財務狀況。

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
from datetime import datetime
creds, _ = default()

gc = gspread.authorize(creds)
import pandas as pd

In [ ]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# 打開你的試算表 (請替換為你的網址)
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1T_CYo5tydoyyBj8LfMpQCcAD4IJqnNkczZb_6WIslZk/edit?gid=0#gid=0')

# 選擇工作表（如果不存在就建立）
try:
    worksheet = gsheets.worksheet('收支紀錄')
except:
    worksheet.append_row(['日期', '項目名稱', '收入', '支出', '請款人', '支付方式', '是否已給錢', '結餘'])

print("--- 輸入收支資料 ---")

--- 輸入收支資料 ---


In [ ]:
while True:
    date_str = input("請輸入日期 (格式 YYYY-MM-DD)：")
    try:
        datetime.strptime(date_str, "%Y-%m-%d")
        break
    except ValueError:
        print("日期格式錯誤，請重新輸入。")

item_str = input("請輸入項目名稱：")

# 收入 / 支出
while True:
    in_or_out = input("這筆是 收入 還是 支出？(輸入 i 或 o)：")
    if in_or_out.lower() == "i":
        income = float(input("請輸入收入金額："))
        expense = 0
        break
    elif in_or_out.lower() == "o":
        expense = float(input("請輸入支出金額："))
        income = 0
        break
    else:
        print("只能輸入 i (收入) 或 o (支出)")

payer = input("請輸入請款人 (沒有可留空)：")
payment_method = input("請輸入支付方式 (現金/信用卡/Line Pay...)：")
paid_status = input("是否已給錢？(是/否)：")

請輸入日期 (格式 YYYY-MM-DD)：2025-09-16
請輸入項目名稱：黏土
這筆是 收入 還是 支出？(輸入 i 或 o)：o
請輸入支出金額：300
請輸入請款人 (沒有可留空)：彭
請輸入支付方式 (現金/信用卡/Line Pay...)：現金
是否已給錢？(是/否)：否


In [ ]:
# Step 3: 新增到收支紀錄分頁（先將結餘欄空著）
new_row = [date_str, item_str, income, expense, payer, payment_method, paid_status, ""]
worksheet.append_row(new_row)
print("\n✅ 已新增到收支紀錄！")
print(new_row)


✅ 已新增到收支紀錄！
['2025-09-16', '黏土', 0, 300.0, '彭', '現金', '否', '']


In [ ]:
# Step 4: 讀取所有資料並轉成 DataFrame
records = worksheet.get_all_records()
df = pd.DataFrame(records)
df.columns = df.columns.str.strip()  # 去除欄位空格

# 確保收入、支出是數字
df['收入'] = pd.to_numeric(df.get('收入', 0), errors='coerce').fillna(0)
df['支出'] = pd.to_numeric(df.get('支出', 0), errors='coerce').fillna(0)

In [ ]:
# Step 5: 計算累積結餘
df['結餘'] = (df['收入'] - df['支出']).cumsum()

In [ ]:
# Step 6: 更新結餘欄位回試算表
cell_list = worksheet.range(f"H2:H{len(df)+1}")  # H欄是第8欄
for i, cell in enumerate(cell_list):
    cell.value = int(df.iloc[i]['結餘'])  # 轉成 Python int
worksheet.update_cells(cell_list)
print("✅ 已更新所有結餘欄位")

✅ 已更新所有結餘欄位


In [ ]:
# Step 7: 顯示總結
total_income = df['收入'].sum()
total_expense = df['支出'].sum()
balance = df['結餘'].iloc[-1] if not df.empty else 0

print("\n--- 收支總結 ---")
print(f"總收入: {total_income:,.0f}")
print(f"總支出: {total_expense:,.0f}")
print(f"目前結餘: {balance:,.0f}")


--- 收支總結 ---
總收入: 0
總支出: 300
目前結餘: -300


In [ ]:
# Step 8: 顯示未付款項目（安全檢查）
if '是否已給錢' in df.columns:
    unpaid = df[df['是否已給錢'].str.contains("否", na=False)]
    if not unpaid.empty:
        print("\n--- 未付款項目 ---")
        print(unpaid[['日期', '項目名稱', '支出', '請款人', '支付方式']])
    else:
        print("\n✅ 沒有未付款項目")
else:
    print("\n❌ 試算表沒有「是否已給錢」欄位")


--- 未付款項目 ---
           日期 項目名稱   支出 請款人 支付方式
0  2025-09-16   黏土  300   彭   現金
